RNN based Password Manager :- 
1. Recurrent Neural Networks (RNNs)
RNNs, particularly Long Short-Term Memory (LSTM) networks, are effective for sequence prediction tasks, including password generation. They can learn the statistical properties of strong passwords from a large corpus of existing passwords and generate new ones that adhere to those properties.

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding

# Load and preprocess the dataset
def load_data(file_path):
    with open(file_path, 'r') as file:
        text = file.read()
    return text

def preprocess_data(text):
    chars = sorted(list(set(text)))
    char_to_index = {c: i for i, c in enumerate(chars)}
    index_to_char = {i: c for i, c in enumerate(chars)}
    return chars, char_to_index, index_to_char

def create_sequences(text, char_to_index, seq_length=40):
    sequences = []
    next_chars = []
    for i in range(0, len(text) - seq_length):
        sequences.append(text[i: i + seq_length])
        next_chars.append(text[i + seq_length])
    X = np.zeros((len(sequences), seq_length, len(char_to_index)), dtype=np.bool)
    y = np.zeros((len(sequences), len(char_to_index)), dtype=np.bool)
    for i, sequence in enumerate(sequences):
        for t, char in enumerate(sequence):
            X[i, t, char_to_index[char]] = 1
        y[i, char_to_index[next_chars[i]]] = 1
    return X, y

# Build the model
def build_model(seq_length, num_chars):
    model = Sequential([
        LSTM(128, input_shape=(seq_length, num_chars)),
        Dense(num_chars, activation='softmax')
    ])
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model

# Generate passwords
def generate_password(model, char_to_index, index_to_char, seq_length=40, num_chars=12):
    start_index = np.random.randint(0, len(text) - seq_length - 1)
    generated = ''
    sentence = text[start_index: start_index + seq_length]
    for i in range(num_chars):
        x_pred = np.zeros((1, seq_length, len(char_to_index)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_to_index[char]] = 1
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = np.argmax(preds)
        next_char = index_to_char[next_index]
        generated += next_char
        sentence = sentence[1:] + next_char
    return generated

# Example usage
file_path = 'passwords.txt'  # Path to your dataset
text = load_data(file_path)
chars, char_to_index, index_to_char = preprocess_data(text)
X, y = create_sequences(text, char_to_index)
model = build_model(X.shape[1], len(char_to_index))
model.fit(X, y, batch_size=128, epochs=20)
print(generate_password(model, char_to_index, index_to_char))

FileNotFoundError: [Errno 2] No such file or directory: 'passwords.txt'